In [2]:
import pandas as pd
import numpy as np
import random as rd
import sklearn as sk

class ProcessingData:
    @staticmethod
    def ShuffleData(df: pd.DataFrame):
        for i in range(len(df)):
            rand_i=rd.randint(0,len(df)-1)
            #zamiana miejscami liczb
            df.loc[i],df.loc[rand_i] = df.loc[rand_i],df.loc[i]
        return df
    @staticmethod
    def SplitData(df:pd.DataFrame,x:int,y:int):
        #walidacja
        if x+y>100 or x<0 or y<0 or x+y<100:
            print("Podano zbyt dużą/małą ilość danych do podziału")
            return
        train_set,test_set = pd.DataFrame(),pd.DataFrame()
        for i in range(len(df)):
            #uwarunkowanie podzialu
            if rd.random()<x/100:
                train_set = train_set.append(df.loc[i])
            else:
                test_set = test_set.append(df.loc[i])
        return train_set,test_set
    @staticmethod
    def NormalizeData(df:pd.DataFrame):
        for atributte in df.columns[:-1]:
            mean = df[atributte].mean()
            stddev = df[atributte].std()
            #wyliczenie normy
            df[atributte] = (df[atributte]-mean)/stddev
        return df



In [3]:
class softClassificator:
    @staticmethod
    def fit(df: pd.DataFrame):
        means = {}
        mins = {}
        maxes = {}
        #wpisywanie do slownika wartosci średnich, minimalnych i maksymalnych dla każdego atrybutu
        #z wyjątkiem ostatniego atrybutu, który jest kluczem
        for Class in df.iloc[:,-1].unique():
            means[Class] = {}
            mins[Class] = {}
            maxes[Class] = {}
            for atributte in df.columns:
                if atributte != df.columns[-1]:
                    means[Class][atributte] = df[df.iloc[:,-1] == Class][atributte].mean()
                    mins[Class][atributte] = df[df.iloc[:,-1] == Class][atributte].min()
                    maxes[Class][atributte] = df[df.iloc[:,-1] == Class][atributte].max()
                    #przypisanie 1 lub 0 do komorki, gdy wartość atrybutu jest w przedziale
                    for row in df.index:
                        if df.loc[row,atributte] < means[Class][atributte]:
                            df.loc[row,atributte] = 0
                        else:
                            df.loc[row,atributte] = 1
        return df,means,mins,maxes

    @staticmethod
    def predict_sample(sample:pd.Series, means:dict, mins:dict, maxes:dict) -> str:
        
        #sprawdzenie czy wszystkie atrybuty są w przedziale
        for Class in means.keys():
            for atributte in means[Class].keys():
                if sample[atributte] < means[Class][atributte]:
                    sample[atributte] = 0
                else:
                    sample[atributte] = 1
        max_probability = 0
        max_Class = ""
        #obliczenie prawdości dla każdej klasy i wybór klasy z największą prawdopodobieństwem
        for Class in means.keys():
            probability = 1
            #prawdopodobieństwo dla każdego atrybutu liczone jest jako iloczyn prawdopodobieństwa dla atrybutu i klasy
            for atributte in means[Class].keys():
                if sample[atributte] == 0:
                    probability *= (1-means[Class][atributte])
                else:
                    probability *= means[Class][atributte]
            #jeśli prawdopodobieństwo jest większe od największego, zapamiętanie klasy i prawdopodobieństwa
            if probability > max_probability:
                max_probability = probability
                max_Class = Class
        return max_Class

    @staticmethod
    def accuracy(df: pd.DataFrame, means:dict, mins:dict, maxes:dict) -> float:
        correct = 0

        # sprawdzenie poprawnosci dla każdego przykładu w stosunku do wszystkich klas
        for row in df.index:
            if df.loc[row,df.columns[-1]] == softClassificator.predict_sample(df.loc[row],means,mins,maxes):
                correct += 1
        return f"{round(correct/len(df) * 100,2)}%"


In [19]:
for i in range(5):
    seedsDF = pd.read_csv("seeds.csv",sep=",")
    seedsDF = ProcessingData.ShuffleData(seedsDF)
    seedsDF = ProcessingData.NormalizeData(seedsDF)
    train_set,test_set = ProcessingData.SplitData(seedsDF,80,20)
    train_set,means,mins,maxes = softClassificator.fit(train_set)
    print(softClassificator.accuracy(test_set,means,mins,maxes))


52.78%
42.86%
47.06%
42.86%
56.41%
